# Debug notebook
This notebook is only used to troubleshoot the S2 ARD workflow processor.

## S2 ARD workflow

In [9]:
import json
from utils.prepLS import prepareLS

def process_scene(json_data):
    loaded_json = json.loads(json_data)
    #for x in loaded_json:
    #    print("%s: %s" % (x, loaded_json[x]))
    prepareLS(**loaded_json)

## S2 ARD job queue processor

In [10]:
import rediswq

host="redis-master"
# Uncomment next two lines if you do not have Kube-DNS working.
# import os
# host = os.getenv("REDIS_SERVICE_HOST")

q = rediswq.RedisWQ(name="jobLS", host=host)
print("Worker with sessionID: " +  q.sessionID())
print("Initial queue state: empty=" + str(q.empty()))

Worker with sessionID: f711dc19-6e28-4daa-8393-a53703c153fa
Initial queue state: empty=False


In [11]:
while not q.empty():
    item = q.lease(lease_secs=1800, block=True, timeout=600)
    if item is not None:
        itemstr = item.decode("utf=8")
        print("Working on " + itemstr)
        #time.sleep(10) # Put your actual work here instead of sleep.
        process_scene(itemstr)
        q.complete(item)
    else:
        print("Waiting for work")

KeyboardInterrupt: 

In [ ]:
print("Queue empty!")